# IMAGE CREATION POLETTI ET AL.

This program creates the input films for the neural network for the experiments by Poletti et al., except for experiment 2 conditions 7/8.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from microsaccades_functions import *
import sys
import os

plt.switch_backend('agg')

The main parameters, also read in from the command line.

`vel_on`: linear motion with 30arcmin/s off/on.
`rem_on`: fixatonal eye movements off/on.
`rect`: frame needed? (exp 2).
`cem_on`: center movements, if on: frame and dot moved according to fixational EM.


$\sigma$: standard deviation of fEM

In [ ]:
#---------------------------------------------------------------------------------------IMAGE-CREATION

image_size = 280 #30 #280
image_height = 30 #280 #120
radius = 4. #dot size of 4arcmin -> 8px
rect_size = 110. #size of inner radius 
vel = 0.06 #velocity is 30 arcmin/s -> 60px/1000ms

suff = sys.argv[1]
vel_on = int(sys.argv[2])
angle = sys.argv[3]
rem_on = int(sys.argv[4])
rect = int(sys.argv[5])
cem_on = int(sys.argv[6])

#for 1D
angle = 0. #horizontal
direction = float(angle)*np.pi/8. #dot moving direction in arc

#for normal distributed microsaccades
sigma = 0.447

file_location = "/video/img_input/poletti2010/"+str(suff)

film_length = 600 #int(framerate)*(stripe_width+gap)

Create and save random walk (in order to compare positions with results later on)

In [ ]:
#get normal 2d distribution
#use this to get same statistics as for actual 2D random walk -> just take distance 
mean = [0, 0]
cov = [[1, 0], [0, 1]]
tl_x, tl_y = sigma*np.random.multivariate_normal(mean, cov, film_length).T

#in 1D, use only x-component
rm = tl_x #np.sign(tl_x)*np.sqrt(tl_x*tl_x+tl_y*tl_y)

d_data = open(file_location+'/displacement.data','w+')
np.save(d_data, (tl_x, tl_y))
d_data.close()

In [ ]:
#initial position
pos = (image_height/2.-0.5, image_size/2.-0.5)
center = (image_height/2.-0.5,image_size/2.-0.5) #for reference frame

This loop creates the primary images. Depending on the chosen options, different parts are included/left out in the images.

In [ ]:
for f in range(film_length):
    canvas = np.zeros((image_height, image_size))
    
    #update position
    #for dot moving task-----------------------
    if vel_on==1:
        pos = (pos[0]+vel*np.sin(direction),pos[1]+vel*np.cos(direction))
    #------------------------------------------
    
    #for random eye movements------------------
    #REPLACED FOR 1D!
    if rem_on==1:
        #pos = (pos[0]+tl_y[f],pos[1]+tl_x[f])
        pos = (pos[0],pos[1]+rm[f])
    #------------------------------------------
    
    #for randomly moving rectangle frame-------
    if cem_on==1:
        #center = (center[0]+tl_y[f],center[1]+tl_x[f])
        center = (center[0],center[1]+rm[f])
    #------------------------------------------
    
    for i in range(image_height):
        for j in range(image_size):
            #get the distance to the current dot position
            dist = np.sqrt((float(i)-pos[0])*(float(i)-pos[0])+(float(j)-pos[1])*(float(j)-pos[1]))
            #2arcmin (4px) is the radius of the dot. The transition is smooth
            if dist <= 3.5:
                canvas[i,j] = 1
            elif 3.5 < dist and dist < 4.5:
                canvas[i,j] = 4.5 - dist
            #rectangle-(exp2)--------------------------
            if rect==1:
                if abs(float(i)-center[0]) > rect_size or abs(float(j)-center[1])> rect_size:
                    canvas[i,j] = 1
            #------------------------------------------
     

Plot the output.

In [ ]:
    fig = plt.figure()
    #fig.set_size_inches(0.25,1)
    fig.set_size_inches(2,30./140.)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(canvas, cmap='gray')
    
    plt.savefig(file_location + "/first"+str(f+1).zfill(3)+".png",  dpi = 140) #120)
    plt.close()
    
    img = cv2.imread(file_location + "/first"+str(f+1).zfill(3)+".png",0)
    rows,cols = img.shape

Finally, add the Gaussian blur to the image and save it.

In [ ]:
    #--------------------------------------------------------------------------------GAUSSIAN-BLURRING
    img = cv2.imread(file_location + "/first"+str(f+1).zfill(3)+".png",0)
    rows,cols = img.shape
    
    blur = cv2.GaussianBlur(img,(3,3),0.5)
    
    fig = plt.figure()
    fig.set_size_inches(2,30./140.)
    #fig.set_size_inches(0.25,1)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(blur, cmap='gray')
    
    #be careful with image dimensions
    plt.savefig(file_location + "/second"+str(f+1).zfill(3)+".png",  dpi = int(image_size/2.))
    plt.close()